In [1]:
# AutoEncoder를 이용한 MNIST Reconstruction
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [5]:
# MNIST 데이터 다운로드
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/",one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [6]:
learning_rate = 0.02
training_epochs = 50   # 반복횟수
batch_size = 256       # 배치개수
display_step = 1       # 손실함수 출력 주기
examples_to_show = 10  # MNIST Reconstruction
input_size = 784       # 28*28
hidden1_size = 256
hidden2_size = 128

In [7]:
# Autoencoder는 Unsupervised Learning이므로 타겟 레이블(label) y가 필요하지 않다.
x = tf.placeholder(tf.float32, shape=[None, input_size])

In [18]:
# Autoencoder 구조 정의
def build_autoencoder(x):
    # encoding = 784 -> 256 ->128
    W1 = tf.Variable(tf.random_normal(shape=[input_size, hidden1_size]))
    b1 = tf.Variable(tf.random_normal(shape=[hidden1_size]))
    H1_output = tf.nn.sigmoid(tf.matmul(x, W1) + b1) 
    W2 = tf.Variable(tf.random_normal(shape=[hidden1_size, hidden2_size]))
    b2 = tf.Variable(tf.random_normal(shape=[hidden2_size]))
    H2_output = tf.nn.sigmoid(tf.matmul(H1_output, W2) + b2)
    # decoding = 128 -> 256 -> 784
    W3 = tf.Variable(tf.random_normal(shape=[hidden2_size, hidden1_size]))
    b3 = tf.Variable(tf.random_normal(shape=[hidden1_size]))
    H3_output = tf.nn.sigmoid(tf.matmul(H2_output, W3) + b3)
    W4 = tf.Variable(tf.random_normal(shape=[hidden1_size, input_size])) 
    b4 = tf.Variable(tf.random_normal(shape=[input_size]))
    reconstructed_x = tf.nn.sigmoid(tf.matmul(H3_output, W4) + b4)
    
    return reconstructed_x

In [19]:
# Autoencoder 선언
y_pred = build_autoencoder(x)
y_true = x   # 타겟데이터 = inputdata

# 손실함수와 옵티마이저 정의
loss = tf.reduce_mean(tf.pow(y_true - y_pred, 2))  # MSE(Mean of Squared Error)
train_step = tf.train.RMSPropOptimizer(learning_rate).minimize(loss)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(training_epochs):
        total_batch = int(mnist.train.num_examples/batch_size)
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            _, current_loss = sess.run([train_step, loss], feed_dict={x: batch_xs})
        if epoch % display_step == 0:
            print("반복(Epoch): %d, 손실 함수(Loss): %f" % ((epoch+1), current_loss))
            
    # Reconstruction 수행
    reconstructed_result = sess.run(y_pred, feed_dict={x: mnist.test.images[:examples_to_show]})
    # 원본 MNIST 데이터와 Reconstruction 결과 비교
    f, a = plt.subplots(2, 10, figsize=(10, 2))
    for i in range(examples_to_show):
        a[0][i].imshow(np.reshape(mnist.test.images[i], (28, 28)))
        a[1][i].imshow(np.reshape(reconstructed_result[i], (28, 28)))
    f.savefig('reconstructed_mnist_image.png')
    f.show()

반복(Epoch): 1, 손실 함수(Loss): 0.199560
반복(Epoch): 2, 손실 함수(Loss): 0.130786
반복(Epoch): 3, 손실 함수(Loss): 0.112909
반복(Epoch): 4, 손실 함수(Loss): 0.105110
반복(Epoch): 5, 손실 함수(Loss): 0.102791
반복(Epoch): 6, 손실 함수(Loss): 0.100875
반복(Epoch): 7, 손실 함수(Loss): 0.092085
반복(Epoch): 8, 손실 함수(Loss): 0.086201
반복(Epoch): 9, 손실 함수(Loss): 0.084023
반복(Epoch): 10, 손실 함수(Loss): 0.083440
반복(Epoch): 11, 손실 함수(Loss): 0.077640
반복(Epoch): 12, 손실 함수(Loss): 0.073768
반복(Epoch): 13, 손실 함수(Loss): 0.075016
반복(Epoch): 14, 손실 함수(Loss): 0.071974
반복(Epoch): 15, 손실 함수(Loss): 0.069947
반복(Epoch): 16, 손실 함수(Loss): 0.068122
반복(Epoch): 17, 손실 함수(Loss): 0.064921
반복(Epoch): 18, 손실 함수(Loss): 0.064196
반복(Epoch): 19, 손실 함수(Loss): 0.063992
